In [1]:
import tweepy
import requests as r
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import csv
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from twitter_keys import Access_Token, Access_Token_Secret, Consumer_Key, Consumer_API_Secret

In [2]:
auth = tweepy.OAuthHandler(Consumer_Key, Consumer_API_Secret)
auth.set_access_token(Access_Token, Access_Token_Secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [ ]:
# IndexError: list index out of range

In [10]:
#Return last tweet from selected user.

last_tweet = api.search('@grantaguinaldo')

last_tweet_text = last_tweet['statuses'][0]['text'].lower()
last_tweet_id = last_tweet['statuses'][0]['id']
last_tweet_screen_name = last_tweet['statuses'][0]['user']['screen_name']

twitter_search_name = re.findall(r"(@[a-z0-9\S]+)", last_tweet_text)
last_tweet_analysis = twitter_search_name[1]

list_ = [last_tweet_text, last_tweet_id, last_tweet_screen_name, last_tweet_analysis]
list_

['@grantaguinaldo @enveraconsult @jetblue i’m impressed!',
 950634855561220096,
 'MelanyeT',
 '@enveraconsult']

In [13]:
#Check screen name against file for duplicates.
with open('./data/twitter_log.csv', mode = 'r') as f:
    reader = csv.reader(f, delimiter = ',')

    for row in reader:
        
        if last_tweet_analysis in row[3]:
            result = True
        
        else:
            result = False

In [6]:
#Write data to CSV if result == False.

with open('./data/twitter_log.csv', mode = 'a') as f:
    writer = csv.writer(f)
    writer.writerow([last_tweet_text, last_tweet_id, last_tweet_screen_name, last_tweet_analysis])

In [ ]:
tweet_text = []
for x in range(25):
    public_tweets = api.user_timeline(twitter_search_name[1], page = x)

    for tweet in public_tweets:
        tweet_text.append(tweet['text'])

In [ ]:
compound_score = []
analyzer = SentimentIntensityAnalyzer()
for text_ in tweet_text:
    score = analyzer.polarity_scores(text_)['compound']
    compound_score.append(score)

plt.figure(figsize = (10, 10))
plt.plot(np.arange(1, len(compound_score) + 1), compound_score, marker='o', alpha = 0.5, label = twitter_search_name[1])
plt.xlabel('Tweets Ago', fontsize = 14)
plt.ylabel('Compound Score (unitless)', fontsize = 14)
plt.title('Tweet Sentiment Over Last 500 Tweets', fontsize = 14)
plt.ylim(-1, 1, 0.25)
plt.legend(title="Tweets")
plt.savefig('tweet_sentiment.png')
plt.show()

#     status_text = 'Thanks for the tweet ' + '@' + last_tweet_screen_name + ' heres the analysis of ' + twitter_search_name[1]
#     api.update_with_media('/Users/gta/dev/hw-7/tweet_sentiment.png', status = status_text, in_reply_to_status_id = last_tweet_id)

In [ ]:
# try:
#     tweet_extract_post()
#     tweet_analysis()
# except:
#     print('Sorry, cannot analyze twitter name')